In [13]:
import pandas as pd
import json

In [ ]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/wangrongsheng/ag_news/" + splits["test"])


In [12]:
# World (0), Sports (1), Business (2), Sci/Tech (3).

df['label'].value_counts()

label
2    30000
3    30000
1    30000
0    30000
Name: count, dtype: int64

In [22]:
# Carica il file JSON come JSON Lines
df = pd.read_json('dataset.json', lines=True)


In [17]:
# Calcola la lunghezza di ogni stringa nella colonna 'short_description'
df['short_description_length'] = df['short_description'].apply(len)

# Trova la lunghezza massima e minima
max_length = df['short_description_length'].max()
min_length = df['short_description_length'].min()

print(f"Lunghezza massima: {max_length}")
print(f"Lunghezza minima: {min_length}")


Lunghezza massima: 1472
Lunghezza minima: 0


In [20]:
len(df)

209527

In [19]:
# Filtra le righe in cui la lunghezza della colonna 'short_description' è zero
zero_length_rows = df[df['short_description_length'] == 0]

# Mostra i risultati
len(zero_length_rows)


19712

In [21]:
df['category'].value_counts()

category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATI